# Glucose Data Preprocessing for PPGR Analysis

## Introduction
This notebook preprocesses continuous glucose monitoring (CGM), meal, and insulin data to isolate and analyze postprandial glucose response (PPGR) events. The workflow includes data import, event alignment, filtering, and calculation of net glucose responses after accounting for insulin action. Each step is annotated with concise explanations to clarify the logic and methodology.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.table import Table
import seaborn as sns
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import datetime
from datetime import timedelta
from statistics import mode
from scipy.signal import find_peaks


from scipy.integrate import odeint

# Libraries for Correlations
import scipy.stats as stats
from scipy.stats import pearsonr, sem, variation, kruskal,f_oneway
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from scipy.spatial.distance import pdist, squareform

from sklearn.preprocessing import LabelEncoder

from itertools import combinations, permutations

%matplotlib inline


## Imports and Setup

The notebook begins by importing essential libraries for data manipulation (`pandas`, `numpy`), visualization (`matplotlib`, `seaborn`), and scientific computation (`scipy`, `sklearn`). These libraries enable efficient data processing, statistical analysis, and plotting. The `%matplotlib inline` magic ensures plots are rendered within the notebook.

############################################

Processing the Glucose Values to Isolate the PPGR

############################################

## Methodology Overview

The main objective is to identify and extract postprandial glucose response (PPGR) events from the raw dataset. This involves:
- Reading and timestamp-aligning glucose, meal, and insulin data.
- Merging and interleaving meal and bolus (insulin) events to create a unified event timeline.
- Associating each event with the closest glucose measurement and relevant metadata.
- Filtering and structuring the data for further analysis of glucose excursions after meals and insulin dosing.

In [2]:
def read_data(filename):
    unfiltered = pd.read_csv(os.path.join(filename))
    # Use the detected 'MacRoman' encoding
    #unfiltered = pd.read_csv(os.path.join(filename), encoding='MacRoman')
    unfiltered['bg_ts'] = pd.to_datetime(unfiltered['bg_ts'], dayfirst=True, errors='coerce')
    unfiltered['meal_ts'] = pd.to_datetime(unfiltered['meal_ts'], dayfirst=True, errors='coerce')
    unfiltered['bolus_ts'] = pd.to_datetime(unfiltered['bolus_ts'], dayfirst=True, errors='coerce')

    return unfiltered


### Function: read_data

This function loads the participant's data from a CSV file, parses relevant timestamp columns (glucose, meal, bolus), and returns a cleaned DataFrame. This ensures all subsequent processing uses consistent datetime formats.

In [3]:
def interleave_arrays_increasing(meal, bolus):
    """Interleave two sorted arrays in increasing order.

    NOTE: Here, the term 'interleave' refers to the specific process of merging two sequences by alternating their elements.
    """
    result = []
    i, j = 0, 0
    while i < len(meal) and j < len(bolus):
        if meal[i] < bolus[j]:
            result.append(("meal", meal[i]))
            i += 1
        else:
            result.append(("bolus", bolus[j]))
            j += 1
    result.extend([("meal", meal[x]) for x in range(i, len(meal))])
    result.extend([("bolus", bolus[x]) for x in range(j, len(bolus))])
    return result


### Function: interleave_arrays_increasing

This function merges two sorted arrays (meal and bolus event timestamps) into a single timeline, tagging each entry by its source. This is crucial for analyzing the sequence and timing of meal and insulin events relative to glucose measurements.

**def interleave_arrays_increasing(...)**
*Notes from Copilot*

The function `interleave_arrays_increasing` takes two input lists, `meal` and `bolus`, and merges them into a single list called `result`. The merging process is done in increasing order, assuming both input lists are already sorted in ascending order. The function uses two pointers, `i` and `j`, to track the current position in the `meal` and `bolus` lists, respectively.

Inside the `while` loop, the function compares the current elements of `meal` and `bolus`. If the current `meal` value is less than the current `bolus` value, it appends a tuple `("meal", meal[i])` to the result and advances the `meal` pointer. Otherwise, it appends a tuple `("bolus", bolus[j])` and advances the `bolus` pointer. This process continues until one of the lists is fully traversed.

After the loop, there may be remaining elements in either `meal` or `bolus`. The function then extends the `result` list with the remaining elements from each list, tagging them appropriately as either `"meal"` or `"bolus"`. The final output is a single list of tuples, each indicating the source ("meal" or "bolus") and the value, all in increasing order. This approach is similar to the merge step in the merge sort algorithm and is efficient for combining two sorted lists.

### Function: glucoseForMealsTs

Finds the closest glucose measurement for each meal timestamp. This step is essential for associating meal events with the most relevant glucose readings, enabling accurate analysis of postprandial responses.

In [4]:
def glucoseForMealsTs(glucose_ts_array, meal_ts):
    closest_values = []
    for element in meal_ts:
        closest_index = np.abs(glucose_ts_array - element).argmin()
        closest_values.append(glucose_ts_array[closest_index])
    return np.array(closest_values)


### Function: glucoseForEventsTs

For each event (meal or bolus), this function finds the closest glucose timestamp within a 4-hour window. This ensures that only temporally relevant glucose readings are linked to each event, improving the quality of downstream analysis.

In [5]:
def glucoseForEventsTs(glucose_ts_array, events_ts):
    closest_values = []
    for event_type, event_ts in events_ts:
        closest_index = np.abs(glucose_ts_array - event_ts).argmin()
        closest_value = glucose_ts_array[closest_index]

        if abs(closest_value - event_ts) <= pd.Timedelta(hours=4):
            closest_values.append((event_type, event_ts, closest_value))

    return closest_values


In [6]:
def bolusMealSeparation(meal_ts, bolus_start_ts, bolus_dose_filtered):
    viableBolusTimes = []
    for i in range(min(len(bolus_start_ts), len(bolus_dose_filtered))):
        isWithinRange = False
        for j in range(len(meal_ts)):
            if meal_ts[j] - pd.Timedelta(minutes=4) <= bolus_start_ts[i] <= meal_ts[j] + pd.Timedelta(hours=4):
                isWithinRange = True
                break
        if not isWithinRange:
            viableBolusTimes.append((bolus_start_ts[i], bolus_dose_filtered[i]))
    return viableBolusTimes


### Function: bolusMealSeparation

Filters bolus (insulin) events to exclude those that are temporally close to meal events. This helps isolate bolus events that are independent of meals, which is important for distinguishing meal-driven from insulin-driven glucose changes.

In [7]:
def groupBolus2(bolus_array):
    time_ranges = [
        ("6am-10am", datetime.time(6, 0), datetime.time(10, 0)),
        ("10am-2pm", datetime.time(10, 0), datetime.time(14, 0)),
        ("2pm-6pm", datetime.time(14, 0), datetime.time(18, 0)),
        ("6pm-10pm", datetime.time(18, 0), datetime.time(22, 0)),
    ]

    result = []
    df = pd.DataFrame(bolus_array, columns=['Timestamp', 'Value'])
    grouped = df.groupby(df['Timestamp'].dt.date)

    for date, group_data in grouped:
        daily_result = {'Date': date, 'TimeRanges': []}
        for label, start_time, end_time in time_ranges:
            time_mask = (group_data['Timestamp'].dt.time >= start_time) & (group_data['Timestamp'].dt.time < end_time)
            max_value = group_data.loc[time_mask, 'Value'].max()
            max_timestamps = group_data.loc[(time_mask) & (group_data['Value'] == max_value), 'Timestamp'].tolist()
            daily_result['TimeRanges'].append({
                'TimeRange': label,
                'MaxValue': max_value,
                'Timestamps': max_timestamps
            })
        result.append(daily_result)
    return result



### Function: groupBolus2

Groups bolus events by day and predefined time ranges (e.g., morning, afternoon, evening). For each range, it identifies the maximum bolus dose and its timestamps. This categorization supports time-of-day analysis of insulin administration patterns.

In [8]:
def findTimestampsNotCoveredByMeals(result, meal_events):
    time_ranges = {
        "6am-10am": (datetime.time(6, 0), datetime.time(10, 0)),
        "10am-2pm": (datetime.time(10, 0), datetime.time(14, 0)),
        "2pm-6pm": (datetime.time(14, 0), datetime.time(18, 0)),
        "6pm-10pm": (datetime.time(18, 0), datetime.time(22, 0))
    }

    meal_events_set = set(meal_events)
    timestamps_not_covered = []

    for day_result in result:
        for time_range_result in day_result['TimeRanges']:
            time_range_label = time_range_result['TimeRange']
            timestamps = time_range_result['Timestamps']
            time_range_start, time_range_end = time_ranges[time_range_label]

            meal_events_within_range = False
            for meal in meal_events_set:
                meal_time = meal.time()
                time_range_start_datetime = datetime.datetime.combine(day_result['Date'], time_range_start)
                time_range_end_datetime = datetime.datetime.combine(day_result['Date'], time_range_end)

                if time_range_start_datetime <= meal <= time_range_end_datetime:
                    meal_events_within_range = True
                    break

            if not meal_events_within_range:
                timestamps_not_covered.extend(timestamps)
    return timestamps_not_covered


### Function: findTimestampsNotCoveredByMeals

Identifies bolus event timestamps that do not overlap with any meal events within the same time range. This step further refines the set of insulin events to those most likely independent of food intake.

In [9]:
def filter_glucose_levels(glucose_ts_array, glucose_level_array, event_ts):
    start_time = event_ts
    end_time = event_ts + pd.Timedelta(hours=4)
    filtered_glucose_levels = []
    previous_timestamp = None
    for ts, level in zip(glucose_ts_array, glucose_level_array):
        if start_time <= ts <= end_time:
            if previous_timestamp is not None and (ts - previous_timestamp) > pd.Timedelta(minutes=30):
                break
            filtered_glucose_levels.append(level)
            previous_timestamp = ts
    return filtered_glucose_levels


### Function: filter_glucose_levels

Extracts glucose values within a 4-hour window after an event, ensuring no large gaps between measurements. This provides a clean glucose trajectory for each event, suitable for PPGR analysis.

###################################

Read and format DataFrame

###################################

## Data Import and Initial Formatting

This section loads the participant's data, sorts it by glucose timestamp, and extracts relevant columns (glucose, meal, bolus, carbs, tags). This prepares the data for event alignment and further processing.

In [10]:
from pathlib import Path
# Read data from the original file
gpath = Path("D:/RobbieDocuments/ManchesterCSCoordinatedDiabetesStudy/Glucose Data")
participant_file = gpath / 'UoMGlucose2301.csv'  # Update this with the actual file name

insulin_sensitivity_factor = 5.8

unfiltered = read_data(participant_file)
unfiltered.sort_values('bg_ts', inplace=True)
unfiltered.reset_index(drop=True, inplace=True)

bg_ts = pd.to_datetime(unfiltered['bg_ts'].copy().to_numpy(), dayfirst=True, errors='coerce')
glucose_level = unfiltered['glucose_level'].copy().to_numpy()  # Already converted to mmol/L
bolus_ts = pd.to_datetime(unfiltered['bolus_ts'].copy().to_numpy(), dayfirst=True, errors='coerce')
meal_ts = pd.to_datetime(unfiltered['meal_ts'].copy().to_numpy(), dayfirst=True, errors='coerce')

bolus_dose = unfiltered['bolus_dose'].copy().to_numpy()
carbs_g = unfiltered['carbs_g']
meal_tags = unfiltered['meal_tag']
meal_types = unfiltered['meal_Type']


KeyError: 'meal_ts'

## Event Filtering and Alignment

Filters out invalid or missing values from meal and bolus arrays, then aligns meal and bolus events with glucose data. This step produces arrays of event-glucose pairs, ready for feature extraction and further analysis.

In [ ]:

nan_mask = np.isnan(bolus_dose)
bolus_dose_filtered = bolus_dose[~nan_mask]

nat_mask = np.isnat(meal_ts)
meal_ts_filtered = meal_ts[~nat_mask]

nat_mask = np.isnat(bolus_ts)
bolus_start_ts_filtered = bolus_ts[~nat_mask]

closest_glucose_array_meals = glucoseForMealsTs(glucose_level_ts, meal_ts_filtered)

bolusAndValueArray = bolusMealSeparation(meal_ts_filtered, bolus_start_ts_filtered, bolus_dose_filtered)
max_bolus_time_range = groupBolus2(bolusAndValueArray)
bolus_replacement_array = findTimestampsNotCoveredByMeals(max_bolus_time_range, meal_ts_filtered)

interleaved_meal_bolus_array = interleave_arrays_increasing(meal_ts_filtered, bolus_replacement_array)
closest_glucose_meal_bolus_array = glucoseForEventsTs(glucose_level_ts, interleaved_meal_bolus_array)



In [ ]:
# Extract 'carbs_g' and 'meal_tags' from the original data
carbs_g = unfiltered['carbs_g']

data_points = []

for event_type, event_ts, glucose_ts in closest_glucose_meal_bolus_array:
    closest_index = np.abs(glucose_level_ts - glucose_ts).argmin()
    glucose_levels = glucose_level[closest_index:closest_index + 48]

    carbs_value = unfiltered.loc[unfiltered['meal_ts'] == event_ts, 'carbs_g'].values[0] if event_type == "meal" else None
    meal_tag_value = unfiltered.loc[unfiltered['meal_ts'] == event_ts, 'meal_tag'].values[0] if event_type == "meal" else None
    meal_type_value = unfiltered.loc[unfiltered['meal_ts'] == event_ts, 'meal_Type'].values[0] if event_type == "meal" else None
    bolus_dose_value = unfiltered.loc[unfiltered['meal_ts'] == event_ts, 'bolus_dose'].values[0] if event_type == "meal" else None
    bolus_time = unfiltered.loc[unfiltered['meal_ts'] == event_ts, 'bolus_ts'].values[0] if event_type == "meal" else None

    data_point = {
        "EventTimestamp": event_ts,
        "GlucoseLevels": glucose_levels,
        "EventType": event_type,
        "EventTag": carbs_value,
        "MealTag": meal_tag_value,
        "MealType": meal_type_value,
        "BolusTime" : bolus_time,
        "BolusDose" : bolus_dose_value
    }
    data_points.append(data_point)


## Building the Event-Feature Dataset

For each event (meal or bolus), this section extracts a window of glucose values and relevant metadata (carbs, meal type, bolus dose, etc.). The result is a structured list of event-feature dictionaries, which is then converted into a DataFrame for analysis.

In [ ]:
# Assume data_points is already defined as your input DataFrame
GlucoseEvents_exploded_clean = pd.DataFrame(data_points)


## Feature Engineering and Categorization

Adds new columns to the event-feature DataFrame, such as day of the week, hour, and meal category. This enables stratified analysis of PPGR by time and meal type, even when explicit meal labels are missing.

In [ ]:

GlucoseEvents_exploded_clean['EventTimestamp'] = pd.to_datetime(GlucoseEvents_exploded_clean['EventTimestamp'])
GlucoseEvents_exploded_clean['day_of_the_week'] = GlucoseEvents_exploded_clean['EventTimestamp'].dt.dayofweek
GlucoseEvents_exploded_clean['hour'] = GlucoseEvents_exploded_clean['EventTimestamp'].dt.hour

# Ensure Meal_Type is prioritized for MealCategory
GlucoseEvents_exploded_clean['MealCategory'] = GlucoseEvents_exploded_clean['MealType']

# Assign time-based categories where MealType is not available
time_based_categories = pd.cut(
    GlucoseEvents_exploded_clean['hour'],
    bins=[0, 10, 16, 22],
    labels=['Breakfast', 'Lunch', 'Dinner'],
    right=False
)

GlucoseEvents_exploded_clean['MealCategory'].fillna(time_based_categories, inplace=True)


## Inspecting the Event-Feature DataFrame

Displays the processed DataFrame to verify that all features and event alignments are correct before further analysis. This step is useful for debugging and validation.

In [ ]:
# Print the DataFrame
print(GlucoseEvents_exploded_clean)



## Modeling Insulin Action and Calculating Net Response

Defines a function to model the effect of rapid-acting insulin on glucose levels, then applies it to each event. The resulting 'NetResponse' column represents glucose trajectories adjusted for insulin action, isolating the meal-driven component of PPGR.

In [ ]:
# Function to compute insulin action profile for rapid-acting insulin
def insulin_action_profile(time, bolus_time, insulin_dose, insulin_sensitivity):
    insulin_effect = np.zeros_like(time, dtype=float)

    for i, t in enumerate(time):
        if t >= bolus_time + 15 and t < bolus_time + 240:
            if t <= bolus_time + 60:
                insulin_effect[i] = -1 * (t - bolus_time - 15) * (insulin_dose * insulin_sensitivity / 45)  # onset to peak
            else:
                insulin_effect[i] = -1 * (insulin_dose * insulin_sensitivity - ((t - bolus_time - 60) * (insulin_dose * insulin_sensitivity / 180)))  # peak to end

    return insulin_effect


# Initialize an empty list to store net responses
net_responses = []

# Iterate over each row in the GlucoseEvents DataFrame
for index, row in GlucoseEvents_exploded_clean.iterrows():
    # Get the necessary parameters for insulin action
    bolus_time = row['BolusTime']  # This should be a pandas Timestamp
    bolus_dose = row['BolusDose']
    glucose_levels = row['GlucoseLevels']

    # Remove non-numeric glucose levels (e.g., 'Low', 'High', etc.)
    cleaned_glucose_levels = []
    for glucose in glucose_levels:
        try:
            cleaned_glucose_levels.append(float(glucose))  # Try converting to float
        except ValueError:  # If it fails, skip or handle accordingly
            cleaned_glucose_levels.append(np.nan)  # Replace non-numeric with NaN (or handle as needed)

    # Convert the list of glucose levels to a numpy array for calculations
    glucose_levels = np.array(cleaned_glucose_levels)

    # Convert BolusTime to minutes since the start of the observation
    start_time = row['EventTimestamp']  # Reference time (event time)
    bolus_time_in_minutes = (bolus_time - start_time).total_seconds() / 60.0  # Convert to minutes

    # Create a time vector based on the length of glucose_levels
    time = np.arange(len(glucose_levels)) * 5  # Assuming glucose levels are recorded every 5 minutes

    # Calculate the insulin effect based on the bolus dose and time
    insulin_effect = insulin_action_profile(time, bolus_time_in_minutes, bolus_dose, insulin_sensitivity_factor)

    # Calculate net response by adjusting glucose levels for insulin effect
    net_response = glucose_levels + insulin_effect  # Element-wise addition

    # Store the net response
    net_responses.append(net_response)

# Add the net responses to the GlucoseEvents DataFrame
GlucoseEvents_exploded_clean['NetResponse'] = net_responses

# Display the updated DataFrame
print(GlucoseEvents_exploded_clean[['EventTimestamp', 'GlucoseLevels', 'BolusTime', 'BolusDose', 'NetResponse']])



## Final Output and Next Steps

The processed DataFrame now contains event-aligned glucose trajectories, event metadata, and net responses adjusted for insulin action. This dataset is ready for statistical analysis, visualization, or machine learning to study PPGR patterns and their determinants.

In [ ]:
GlucoseEvents_exploded_clean.head()

## Exporting Processed Data

In [ ]:
# Save the processed DataFrame to a new CSV file
processed_df.to_csv("processed_glucose_data.csv", index=False)